In [1]:
import os
import json
import time
import random
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sentence_transformers import SentenceTransformer
from torch.optim import AdamW


# 1. Custom Dataset 정의
class MathTopicDataset(Dataset):
    def __init__(self, json_dirs, label_mapping):
        self.data = []
        self.label_mapping = label_mapping

        # 데이터 로드
        for json_dir in json_dirs:
            for root, _, files in os.walk(json_dir):
                for file in files:
                    if file.endswith(".json"):
                        filepath = os.path.join(root, file)
                        with open(filepath, "r", encoding="utf-8") as f:
                            item = json.load(f)
                            self.data.append(
                                {
                                    "text": item["question_text"],
                                    "label": label_mapping[item["question_topic_name"]],
                                }
                            )

        # 데이터 셔플 및 5000개 샘플링
        random.shuffle(self.data)
        if len(self.data) > 5000:
            self.data = self.data[:5000]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]["text"], self.data[idx]["label"]


# 2. 분류 모델 정의
class SentenceTransformerClassifier(nn.Module):
    def __init__(self, pretrained_model_name, num_labels):
        super(SentenceTransformerClassifier, self).__init__()
        self.encoder = SentenceTransformer(pretrained_model_name)
        self.classifier = nn.Linear(
            self.encoder.get_sentence_embedding_dimension(), num_labels
        )

    def forward(self, sentences):
        embeddings = self.encoder.encode(sentences, convert_to_tensor=True)
        logits = self.classifier(embeddings)
        return logits


# 3. 학습 코드
def main():
    # 설정
    train_json_dirs = [
        "D:/programming/python/chunjae/finalproject/images/processed_data/training_st/elementary3",
        "D:/programming/python/chunjae/finalproject/images/processed_data/training_st/elementary4",
        "D:/programming/python/chunjae/finalproject/images/processed_data/training_st/elementary5",
        "D:/programming/python/chunjae/finalproject/images/processed_data/training_st/elementary6",
    ]
    val_json_dirs = [
        "D:/programming/python/chunjae/finalproject/images/processed_data/validation_st/elementary3",
        "D:/programming/python/chunjae/finalproject/images/processed_data/validation_st/elementary4",
        "D:/programming/python/chunjae/finalproject/images/processed_data/validation_st/elementary5",
        "D:/programming/python/chunjae/finalproject/images/processed_data/validation_st/elementary6",
    ]
    batch_size = 32
    num_epochs = 10
    learning_rate = 1e-5
    pretrained_model_name = (
        "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    )
    checkpoint_path = "best_model_checkpoint.pth"  # 체크포인트 저장 경로

    # 라벨 매핑 생성
    label_mapping = {}
    for json_dir in train_json_dirs:
        for root, _, files in os.walk(json_dir):
            for file in files:
                if file.endswith(".json"):
                    filepath = os.path.join(root, file)
                    with open(filepath, "r", encoding="utf-8-sig") as f:
                        item = json.load(f)
                        label_mapping[item["question_topic_name"]] = label_mapping.get(
                            item["question_topic_name"], len(label_mapping)
                        )

    print(f"총 라벨 수: {len(label_mapping)}")

    # Dataset 및 DataLoader 초기화
    train_dataset = MathTopicDataset(train_json_dirs, label_mapping)
    val_dataset = MathTopicDataset(val_json_dirs, label_mapping)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # 모델 초기화
    device = torch.device(
        "cuda:1" if torch.cuda.is_available() else "cpu"
    )  # NVIDIA GPU 사용
    model = SentenceTransformerClassifier(pretrained_model_name, len(label_mapping)).to(
        device
    )

    # 디바이스 정보 출력
    print(f"Using device: {device}")
    if next(model.parameters()).is_cuda:
        print("Model is on GPU")
    else:
        print("Model is on CPU")

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    best_val_loss = float("inf")  # 최상의 검증 손실 초기화

    # 학습 루프
    for epoch in range(num_epochs):
        start_time = time.time()  # 시작 시간 기록
        model.train()
        train_loss = 0.0
        for i, (texts, labels) in enumerate(train_loader):
            labels = labels.to(device)
            logits = model(texts)
            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            # 진행 상황 및 남은 시간 출력
            elapsed_time = time.time() - start_time
            estimated_total_time = elapsed_time / (i + 1) * len(train_loader)
            estimated_remaining_time = estimated_total_time - elapsed_time
            print(
                f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], "
                f"Elapsed Time: {elapsed_time:.2f}s, Estimated Remaining Time: {estimated_remaining_time:.2f}s",
                end="\r",
            )

        # 검증 루프
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for texts, labels in val_loader:
                labels = labels.to(device)
                logits = model(texts)
                loss = criterion(logits, labels)
                val_loss += loss.item()

                _, predicted = torch.max(logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # 최상의 모델 체크포인트 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), checkpoint_path)
            print(f"\n체크포인트 저장됨: {checkpoint_path}")

        # 로그 출력
        print(
            f"\nEpoch [{epoch + 1}/{num_epochs}], "
            f"Train Loss: {train_loss / len(train_loader):.4f}, "
            f"Validation Loss: {val_loss / len(val_loader):.4f}, "
            f"Validation Accuracy: {100 * correct / total:.2f}%"
        )

    # 최종 모델 저장
    torch.save(model.state_dict(), "sentence_transformer_classifier.pth")
    print("모델 학습 완료 및 저장됨.")


if __name__ == "__main__":
    main()

c:\Users\seong\anaconda3\envs\kst_pp\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\seong\anaconda3\envs\kst_pp\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you m

'NoneType' object has no attribute 'cadam32bit_grad_fp32'
총 라벨 수: 319
Using device: cpu
Model is on CPU


KeyboardInterrupt: 

In [1]:
import torch
print(torch.cuda.is_available())  # GPU 사용 가능 여부
print(torch.cuda.device_count())  # 사용 가능한 GPU 수
print(torch.cuda.current_device())  # 현재 활성화된 GPU
print(torch.cuda.get_device_name(0))  # GPU 이름

False
0


AssertionError: Torch not compiled with CUDA enabled